In [14]:
import onnxruntime as ort
print(ort.get_device()) # 'GPU'가 나오면 성공!
print(ort.get_available_providers())

GPU
['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [1]:
import torch
import torch.nn as nn
import timm
import os


/home/arkplus/anaconda3/envs/hrnet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 설정
NUM_KEYPOINTS = 4
DEVICE = torch.device("cpu") # 변환은 CPU에서 해도 충분합니다
INPUT_SIZE = (256, 256)
WEIGHT_PATH = "models/hrnet/hrnet_w32.pth"
ONNX_PATH = "models/onnx/hrnet_w32.onnx"


In [8]:
# 모델
class HRNet(nn.Module):
    def __init__(self, num_keypoints=4, pretrained=True, stride_idx=1):
        super().__init__()
        self.backbone = timm.create_model(
            "hrnet_w32",
            pretrained=pretrained,
            features_only=True
        )
        self.stride_idx = stride_idx
        in_channels = self.backbone.feature_info.channels()[self.stride_idx]

        self.final_layer = nn.Conv2d(in_channels, num_keypoints, kernel_size=1)

    def forward(self, x):
        x = self.backbone(x)[self.stride_idx]
        x = self.final_layer(x)
        return x

In [4]:
# 모델
model = HRNet(num_keypoints=NUM_KEYPOINTS, pretrained=False).to(DEVICE)

try:
    state_dict = torch.load(WEIGHT_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)
    model.eval()
    print("모델 로드 성공")
except Exception as e:
    print(f"모델 로드 실패: {e}")
    exit()

# 더미 입력 생성 (모델이 예상하는 입력 크기: B, C, H, W)
dummy_input = torch.randn(1, 3, INPUT_SIZE[0], INPUT_SIZE[1], device=DEVICE)


모델 로드 성공


In [5]:
print(dummy_input.shape)

torch.Size([1, 3, 256, 256])


In [11]:
# export
torch.onnx.export(
    model,                      # 실행할 모델
    dummy_input,                # 더미 입력
    ONNX_PATH,                  # 저장할 경로
    export_params=True,         # 가중치 포함 여부
    opset_version=18,           # ONNX 버전 (11 또는 12 권장)
    do_constant_folding=True,   # 상수 폴딩 최적화
    input_names=['input'],      # 입력 노드 이름
    output_names=['output'],    # 출력 노드 이름
    dynamic_axes={              # 배치 크기가 변경될 수 있음을 명시
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)
print(f"ONNX 변환 완료: {ONNX_PATH}")


/tmp/ipykernel_10680/3856616019.py:2: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0219 10:00:38.079000 10680 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0219 10:00:38.079000 10680 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0219 10:00:38.080000 10680 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size

[torch.onnx] Obtain model graph for `HRNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `HRNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/home/arkplus/anaconda3/envs/hrnet/lib/python3.10/copyreg.py:101: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 592 of general pattern rewrite rules.
ONNX 변환 완료: models/onnx/hrnet_w32.onnx
